# Spectrum xAI

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm

c:\Users\ismoi\anaconda3\envs\ai\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
CLASS = 1
IMAGE_PATH = f"data/dataset/Class{CLASS}/"

In [3]:
class DAGMDataset(Dataset):
    def __init__(self, root_dir, split="Train", transform=None):
        """
        Args:
            root_dir (str): Chemin du dossier dataset.
            split (str): "Train" ou "Test".
            transform (callable, optional): Transformation appliquée aux images.
        """
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.data = []
        
        img_dir = os.path.join(root_dir, split)
        label_dir = os.path.join(img_dir, "Label")
        
        images = [f for f in os.listdir(img_dir) if f.endswith((".PNG"))]
        
        for img_name in images:
            img_path = os.path.join(img_dir, img_name)
            label_path = os.path.join(label_dir, img_name.replace(".PNG", "_label.PNG"))
            
            label = 1 if os.path.exists(label_path) else 0
            
            self.data.append((img_path, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, label

In [4]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = DAGMDataset(root_dir=IMAGE_PATH, split="Train", transform=transform)
test_dataset = DAGMDataset(root_dir=IMAGE_PATH, split="Test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 575
Test dataset size: 575


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet121(weights="DEFAULT").to(device)

num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
).to(device)

criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        predicted = (outputs > 0.5).float()
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = 100 * correct_train / total_train
    avg_loss = running_loss / len(train_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}")

100%|██████████| 18/18 [00:12<00:00,  1.49it/s]


Epoch [1/10], Loss: 0.8216, Train Accuracy: 84.6957


100%|██████████| 18/18 [00:10<00:00,  1.67it/s]


Epoch [2/10], Loss: 0.7290, Train Accuracy: 88.0000


100%|██████████| 18/18 [00:10<00:00,  1.66it/s]


...

Epoch [8/10], Loss: 0.6452, Train Accuracy: 100.0000


100%|██████████| 18/18 [00:11<00:00,  1.61it/s]


Epoch [9/10], Loss: 0.6444, Train Accuracy: 100.0000


100%|██████████| 18/18 [00:11<00:00,  1.59it/s]

Epoch [10/10], Loss: 0.6440, Train Accuracy: 100.0000


In [6]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)
        outputs = model(images).squeeze()
        
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 99.65%
